In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
from glob import glob
import random
from tqdm import tqdm
from PIL import Image
import torch_directml

# Verificar si DirectML está disponible
if torch_directml.is_available():
    device = torch_directml.device(0)  # Usar el primer dispositivo (si tienes una GPU compatible)
    print("DirectML está configurado correctamente")
else:
    device = torch.device("cpu")
    print("DirectML no está disponible, usando CPU")

# Definir rutas y parámetros
DATASET_PATH = "Dataset-Flores"
CLASSES = ["astromelia", "cartucho", "lirio", "obispo", "sanjuan"]
IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 50

DirectML está configurado correctamente


In [22]:
def load_sam_model():
    try:
        # Importar SAM
        from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
        
        # Verificar si ya existe el modelo descargado
        model_path = "sam_vit_h_4b8939.pth"
        if not os.path.exists(model_path):
            print("Descargando modelo SAM...")
            import urllib.request
            urllib.request.urlretrieve(
                "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth",
                model_path
            )
        
        print("Cargando modelo SAM...")
        # Usar CPU para SAM ya que DirectML podría no ser compatible directamente
        sam_device = torch.device("cpu")
        sam = sam_model_registry["vit_h"](checkpoint=model_path)
        sam.to(sam_device)
        
        # Crear generador de máscaras automático
        mask_generator = SamAutomaticMaskGenerator(
            model=sam,
            points_per_side=32,  # Más puntos = más segmentaciones detalladas pero más lento
            pred_iou_thresh=0.86,  # Umbral de calidad de predicción 
            stability_score_thresh=0.92,  # Umbral de estabilidad
            crop_n_layers=1,  # Número de capas para recortar la imagen
            crop_n_points_downscale_factor=2,  # Factor de reducción de puntos en recortes
            min_mask_region_area=100,  # Área mínima de segmentación en píxeles
        )
        
        # Crear predictor para puntos específicos (útil si quieres guiar la segmentación)
        predictor = SamPredictor(sam)
        
        print("Modelo SAM cargado correctamente")
        return mask_generator, predictor, True
    
    except ImportError:
        print("ADVERTENCIA: No se pudo importar el módulo segment-anything.")
        print("Por favor, instala SAM con: pip install git+https://github.com/facebookresearch/segment-anything.git")
        return None, None, False
    
    except Exception as e:
        print(f"Error al cargar el modelo SAM: {e}")
        return None, None, False

# Cargar modelo SAM
sam_mask_generator, sam_predictor, sam_available = load_sam_model()

Cargando modelo SAM...
Modelo SAM cargado correctamente


In [23]:
class FloresDataset(Dataset):
    def __init__(self, dataset_path, classes, img_size=128, transform=None, train=True, 
                 sam_mask_generator=None, cache_dir=None):
        self.dataset_path = dataset_path
        self.classes = classes
        self.img_size = img_size
        self.transform = transform
        self.train = train
        self.sam_mask_generator = sam_mask_generator
        self.image_paths = []
        
        # Directorio para cachear máscaras generadas por SAM
        self.cache_dir = cache_dir
        if self.cache_dir and not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir, exist_ok=True)
            print(f"Creado directorio de caché para máscaras: {self.cache_dir}")
        
        print(f"Inicializando dataset {'de entrenamiento' if train else 'de validación'}")
        self.load_data()
        
        if len(self.image_paths) == 0:
            raise ValueError("¡No se encontraron imágenes válidas! Verifica la estructura de carpetas.")
    
    def load_data(self):
        all_image_paths = []
        
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            
            # Verificar si existe la carpeta de clase
            if not os.path.exists(class_path):
                print(f"ADVERTENCIA: La carpeta de clase {class_path} no existe")
                continue
            
            # Verificar subcarpeta de imágenes
            images_path = os.path.join(class_path, "images")
            if not os.path.exists(images_path):
                print(f"ADVERTENCIA: La carpeta de imágenes {images_path} no existe")
                continue
            
            # Obtener todas las imágenes
            image_files = glob(os.path.join(images_path, "*.jpg")) + \
                          glob(os.path.join(images_path, "*.jpeg")) + \
                          glob(os.path.join(images_path, "*.png"))
            
            print(f"Encontradas {len(image_files)} imágenes en la clase {class_name}")
            all_image_paths.extend(image_files)
        
        print(f"Total de imágenes encontradas: {len(all_image_paths)}")
        
        if len(all_image_paths) == 0:
            print("ERROR: No se encontraron imágenes en ninguna de las clases")
            return
        
        # Dividir en entrenamiento y validación
        indices = list(range(len(all_image_paths)))
        np.random.seed(42)  # Para reproducibilidad
        np.random.shuffle(indices)
        split_idx = int(0.8 * len(indices))
        
        if self.train:
            selected_indices = indices[:split_idx]
        else:
            selected_indices = indices[split_idx:]
        
        self.image_paths = [all_image_paths[i] for i in selected_indices]
        print(f"Dataset {'de entrenamiento' if self.train else 'de validación'} creado con {len(self.image_paths)} imágenes")
    
    def generate_mask_with_sam(self, img, img_path):
        """Genera una máscara utilizando SAM o carga desde caché si está disponible"""
        if self.sam_mask_generator is None:
            # Si SAM no está disponible, crear una máscara simple de forma básica
            # Esto es un fallback y no dará buenos resultados
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            _, mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            return mask
        
        # Verificar si existe en caché
        if self.cache_dir:
            img_name = os.path.basename(img_path)
            cache_path = os.path.join(self.cache_dir, f"{os.path.splitext(img_name)[0]}_mask.png")
            
            if os.path.exists(cache_path):
                # Cargar desde caché
                mask = cv2.imread(cache_path, cv2.IMREAD_GRAYSCALE)
                return mask
        
        # Generar máscara con SAM
        masks = self.sam_mask_generator.generate(img)
        
        if not masks:
            # Si SAM no genera máscaras, crear una vacía
            final_mask = np.zeros(img.shape[:2], dtype=np.uint8)
        else:
            # Ordenar por área (de mayor a menor)
            masks = sorted(masks, key=lambda x: x['area'], reverse=True)
            
            # Tomar la máscara más grande como la principal
            largest_mask = masks[0]['segmentation'].astype(np.uint8) * 255
            final_mask = largest_mask
        
        # Guardar en caché si corresponde
        if self.cache_dir:
            img_name = os.path.basename(img_path)
            cache_path = os.path.join(self.cache_dir, f"{os.path.splitext(img_name)[0]}_mask.png")
            cv2.imwrite(cache_path, final_mask)
        
        return final_mask
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        
        # Cargar y preprocesar imagen
        try:
            img = cv2.imread(img_path)
            if img is None:
                raise ValueError(f"No se pudo cargar la imagen: {img_path}")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Generar máscara con SAM o cargar desde caché
            mask = self.generate_mask_with_sam(img, img_path)
            
            # Redimensionar para la red
            img_resized = cv2.resize(img, (self.img_size, self.img_size))
            mask_resized = cv2.resize(mask, (self.img_size, self.img_size))
            
            # Normalizar
            img_normalized = img_resized / 255.0
            mask_normalized = mask_resized / 255.0
            
            # Formato para PyTorch (C, H, W)
            img_tensor = torch.from_numpy(img_normalized.transpose(2, 0, 1)).float()
            mask_tensor = torch.from_numpy(mask_normalized).float().unsqueeze(0)  # Añadir dimensión de canal
            
        except Exception as e:
            print(f"Error al procesar la imagen {img_path}: {e}")
            # Crear tensores vacíos como fallback
            img_tensor = torch.zeros((3, self.img_size, self.img_size), dtype=torch.float32)
            mask_tensor = torch.zeros((1, self.img_size, self.img_size), dtype=torch.float32)
        
        # Aplicar transformaciones si existen
        if self.transform:
            # Crear un diccionario con ambos tensores
            sample = {'image': img_tensor, 'mask': mask_tensor}
            sample = self.transform(sample)
            img_tensor, mask_tensor = sample['image'], sample['mask']
        
        return img_tensor, mask_tensor

# Transformaciones para aumentación de datos
class RandomTransform:
    def __init__(self, p=0.5):
        self.p = p
    
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # Flip horizontal
        if random.random() < self.p:
            image = torch.flip(image, [2])
            mask = torch.flip(mask, [2])
        
        # Flip vertical
        if random.random() < self.p:
            image = torch.flip(image, [1])
            mask = torch.flip(mask, [1])
        
        return {'image': image, 'mask': mask}

# Crear datasets
try:
    # Directorio de caché para máscaras SAM
    cache_dir = "sam_masks_cache"
    
    # Crear transformaciones
    train_transform = RandomTransform(p=0.5)
    
    # Crear datasets
    train_dataset = FloresDataset(
        DATASET_PATH, 
        CLASSES, 
        IMG_SIZE, 
        transform=train_transform, 
        train=True, 
        sam_mask_generator=sam_mask_generator,
        cache_dir=cache_dir
    )
    
    val_dataset = FloresDataset(
        DATASET_PATH, 
        CLASSES, 
        IMG_SIZE, 
        transform=None, 
        train=False, 
        sam_mask_generator=sam_mask_generator,
        cache_dir=cache_dir
    )
    
    # Crear dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    print(f"Datos de entrenamiento: {len(train_dataset)} imágenes")
    print(f"Datos de validación: {len(val_dataset)} imágenes")
    
except Exception as e:
    print(f"Error al crear los datasets: {e}")
    import traceback
    traceback.print_exc()

Inicializando dataset de entrenamiento
Encontradas 400 imágenes en la clase astromelia
Encontradas 400 imágenes en la clase cartucho
Encontradas 400 imágenes en la clase lirio
Encontradas 400 imágenes en la clase obispo
Encontradas 400 imágenes en la clase sanjuan
Total de imágenes encontradas: 2000
Dataset de entrenamiento creado con 1600 imágenes
Inicializando dataset de validación
Encontradas 400 imágenes en la clase astromelia
Encontradas 400 imágenes en la clase cartucho
Encontradas 400 imágenes en la clase lirio
Encontradas 400 imágenes en la clase obispo
Encontradas 400 imágenes en la clase sanjuan
Total de imágenes encontradas: 2000
Dataset de validación creado con 400 imágenes
Datos de entrenamiento: 1600 imágenes
Datos de validación: 400 imágenes


In [24]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        
        # Encoder
        self.down1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.down3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.down4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Bridge
        self.bridge = DoubleConv(512, 1024)
        
        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.up_conv1 = DoubleConv(1024, 512)
        
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up_conv2 = DoubleConv(512, 256)
        
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_conv3 = DoubleConv(256, 128)
        
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_conv4 = DoubleConv(128, 64)
        
        # Output
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # Encoder
        x1 = self.down1(x)
        x = self.pool1(x1)
        
        x2 = self.down2(x)
        x = self.pool2(x2)
        
        x3 = self.down3(x)
        x = self.pool3(x3)
        
        x4 = self.down4(x)
        x = self.pool4(x4)
        
        # Bridge
        x = self.bridge(x)
        
        # Decoder
        x = self.up1(x)
        x = torch.cat([x, x4], dim=1)
        x = self.up_conv1(x)
        
        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.up_conv2(x)
        
        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.up_conv3(x)
        
        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.up_conv4(x)
        
        # Output
        x = self.out(x)
        x = self.sigmoid(x)
        
        return x

# Inicializar modelo
model = UNet(in_channels=3, out_channels=1)
model = model.to(device)

# Función para contar parámetros
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Número de parámetros entrenables: {count_parameters(model):,}")

Número de parámetros entrenables: 31,043,521


In [25]:
# Definir funciones de pérdida y métricas
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.1, verbose=True)

# Función para calcular IoU
def calculate_iou(pred, target):
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    if union < 1e-6:
        return 0
    return (intersection / union).item()

# Función de entrenamiento
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    iou_scores = []
    
    for images, masks in tqdm(dataloader):
        images = images.to(device)
        masks = masks.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Estadísticas
        running_loss += loss.item()
        
        # Calcular IoU
        with torch.no_grad():
            batch_iou = calculate_iou(outputs.detach(), masks)
            iou_scores.append(batch_iou)
    
    epoch_loss = running_loss / len(dataloader)
    epoch_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0
    
    return epoch_loss, epoch_iou

# Función de validación
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    iou_scores = []
    
    with torch.no_grad():
        for images, masks in tqdm(dataloader):
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Estadísticas
            running_loss += loss.item()
            batch_iou = calculate_iou(outputs, masks)
            iou_scores.append(batch_iou)
    
    epoch_loss = running_loss / len(dataloader)
    epoch_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0
    
    return epoch_loss, epoch_iou

# Función para guardar resultados del entrenamiento
def save_training_plots(train_losses, val_losses, train_ious, val_ious):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Entrenamiento')
    plt.plot(val_losses, label='Validación')
    plt.title('Pérdida por época')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_ious, label='Entrenamiento')
    plt.plot(val_ious, label='Validación')
    plt.title('IoU por época')
    plt.xlabel('Época')
    plt.ylabel('IoU')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_results.png')
    plt.show()

In [26]:
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    train_ious = []
    val_ious = []
    patience = 3
    no_improve = 0
    
    for epoch in range(num_epochs):
        print(f"Época {epoch+1}/{num_epochs}")
        
        # Entrenamiento
        train_loss, train_iou = train_epoch(model, train_loader, optimizer, criterion, device)
        train_losses.append(train_loss)
        train_ious.append(train_iou)
        
        # Validación
        val_loss, val_iou = validate(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_ious.append(val_iou)
        
        # Actualizar learning rate
        scheduler.step(val_loss)
        
        print(f"Entrenamiento - Pérdida: {train_loss:.4f}, IoU: {train_iou:.4f}")
        print(f"Validación - Pérdida: {val_loss:.4f}, IoU: {val_iou:.4f}")
        
        # Guardar mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_sam_model.pth")
            print("Guardando mejor modelo...")
            no_improve = 0
        else:
            no_improve += 1
        
        # Early stopping
        if no_improve >= patience:
            print(f"Early stopping después de {patience} épocas sin mejora")
            break
    
    # Guardar modelo final
    torch.save(model.state_dict(), "final_unet_model.pth")
    
    # Guardar gráficas
    save_training_plots(train_losses, val_losses, train_ious, val_ious)
    
    return train_losses, val_losses, train_ious, val_ious

In [27]:
def visualize_predictions(model, dataloader, num_samples=5, device=device):
    model.eval()
    images, masks, predictions = [], [], []
    
    # Obtener algunas muestras
    with torch.no_grad():
        for img, mask in dataloader:
            if len(images) >= num_samples:
                break
            
            img_batch = img.to(device)
            pred_batch = model(img_batch)
            
            # Convertir a NumPy para visualización
            for i in range(min(len(img), num_samples - len(images))):
                images.append(img[i].cpu().numpy())
                masks.append(mask[i].cpu().numpy())
                predictions.append(pred_batch[i].cpu().numpy())
            
            if len(images) >= num_samples:
                break
    
    # Visualizar
    plt.figure(figsize=(15, 5 * num_samples))
    for i in range(num_samples):
        # Imagen original
        plt.subplot(num_samples, 3, i*3 + 1)
        img_display = np.transpose(images[i], (1, 2, 0))  # (C,H,W) -> (H,W,C)
        plt.imshow(img_display)
        plt.title(f"Imagen {i+1}")
        plt.axis('off')
        
        # Máscara real
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(masks[i][0], cmap='gray')
        plt.title(f"Máscara (SAM) {i+1}")
        plt.axis('off')
        
        # Máscara predicha
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(predictions[i][0], cmap='gray')
        plt.title(f"Predicción U-Net {i+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('prediction_examples.png')
    plt.show()

# Cargar el mejor modelo y visualizar
def load_and_visualize():
    if os.path.exists("best_unet_model.pth"):
        model.load_state_dict(torch.load("best_unet_model.pth", map_location=device))
        print("Modelo cargado correctamente")
        
        # Visualizar algunas predicciones
        visualize_predictions(model, val_loader, num_samples=5, device=device)
    else:
        print("No se encontró el modelo guardado")

In [28]:
def predict_on_images(model, input_dir, output_dir="predicciones", device=device):
    model.eval()
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtener todas las imágenes
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(glob(os.path.join(input_dir, f"*{ext}")))
    
    if not image_files:
        print(f"No se encontraron imágenes en {input_dir}")
        return
    
    print(f"Procesando {len(image_files)} imágenes...")
    
    for img_path in tqdm(image_files):
        # Cargar imagen
        img = cv2.imread(img_path)
        if img is None:
            print(f"No se pudo cargar la imagen: {img_path}")
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Guardar tamaño original
        original_size = img.shape[:2]
        
        # Preprocesar
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_tensor = torch.from_numpy(img_resized.transpose(2, 0, 1)).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)  # Añadir dimensión de batch
        
        # Predecir
        with torch.no_grad():
            pred_mask = model(img_tensor)
            pred_mask = pred_mask.squeeze().cpu().numpy()
        
        # Umbralización para binarizar
        pred_binary = (pred_mask > 0.5).astype(np.uint8) * 255
        
        # Redimensionar al tamaño original
        pred_resized = cv2.resize(pred_binary, (original_size[1], original_size[0]))
        
        # Crear superposición
        overlay = img.copy()
        colored_mask = np.zeros_like(img)
        colored_mask[:,:,1] = pred_resized  # Canal verde
        
        # Superponer máscara en imagen original
        alpha = 0.5
        cv2.addWeighted(colored_mask, alpha, overlay, 1 - alpha, 0, overlay)
        
        # Guardar resultados
        base_name = os.path.basename(img_path).split('.')[0]
        
        # Guardar imagen original
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_original.jpg"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        # Guardar máscara predicha
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_mask.png"), pred_resized)
        
        # Guardar superposición
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_overlay.jpg"), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
    
    print(f"Predicciones guardadas en {output_dir}")

In [29]:
# def main():
#     print("Iniciando proceso de segmentación de flores con U-Net y SAM en PyTorch")
    
#     if not sam_available:
#         print("ADVERTENCIA: SAM no está disponible. Las máscaras se generarán de forma básica.")
#         print("Se recomienda instalar SAM para obtener mejores resultados")
    
#     try:
#         # Verificar si tenemos datasets válidos
#         if 'train_loader' not in globals() or 'val_loader' not in globals():
#             print("ERROR: No se pudieron crear los dataloaders correctamente")
#             return
        
#         # Entrenar modelo
#         print("\nIniciando entrenamiento del modelo...")
#         train_losses, val_losses, train_ious, val_ious = train_model(
#             model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10
#         )
        
#         # Visualizar resultados
#         print("\nVisualizando resultados...")
#         load_and_visualize()
        
#         print("\nProceso completado con éxito")
#         print("Para realizar inferencia en nuevas imágenes, use la función predict_on_images()")
#         print("Ejemplo: predict_on_images('carpeta_nuevas_imagenes', 'carpeta_resultados')")
    
#     except Exception as e:
#         print(f"\nERROR durante la ejecución: {e}")
#         import traceback
#         traceback.print_exc()

# if __name__ == "__main__":
#     main()